# Prompts

Langchain has the widest coverage of the possible LLMs you can/might want to use so it is nesessary to be familier with its LLM and Prompt interfaces for ragas.

In [19]:
from langchain.llms import BaseLLM
from langchain.llms.openai import OpenAI
from langchain.chat_models.openai import ChatOpenAI

from rich.pretty import pprint

openaichat = ChatOpenAI()
openai = OpenAI()

In [4]:
openaichat.predict("hai how are you")

"Hello! I'm an AI, so I don't have feelings, but thank you for asking. How can I assist you today?"

In [5]:
openai.predict("hai how are you")

"\n\nI'm doing well, thank you. How about you?"

## Prompts

ref: [docs](https://python.langchain.com/docs/modules/model_io/prompts/)

Prompts are the way we interact with LLMs and Langchain offers 2 components for that:
- Prompt Templates: Parmetrize model inputs
- Example Selectors: Dynamically select examples to include in prompts

### Prompt Templates

In [3]:
from langchain import PromptTemplate

template = """\
You are a naming consultant for new companies.
What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate.from_template(template)
prompt.format(product="colorful socks")

'You are a naming consultant for new companies.\nWhat is a good name for a company that makes colorful socks?\n'

`.from_template()` classmethod is the easiest way to build prompt templates but you can checkout `PromptTemplate` and [Custom Prompt Templates](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/custom_prompt_template.html)

### Chat Prompt Tempalate
LLM services like OpenAI and Anthropic are also now implementing a chat interface on top of LLMs. For a Chat interface you need
- System Message - helps set the behaviour of the assistant.
- list of `HumanMessage` and `AIMessage`. This is the convo your having with the system

In [4]:
from langchain.prompts import (
    ChatPromptTemplate, 
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [7]:
sys_msg = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant that translates {input_language}"
    " to {output_language}"
)
human_msg = HumanMessagePromptTemplate.from_template(
    "{text}"
)


# ChatPromptTemplate has the collection
chat_prompt = ChatPromptTemplate.from_messages([sys_msg, human_msg])
chat_prompt.format_prompt(
    input_language="English", 
    output_language="French", 
    text="I love programming."
).to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

There is also another way that I personally like when creating `ChatPromptTemplate` is directly from the `BaseMessage` objects like `AIMessage`, `HumanMessage`, `SystemMessage`. This is usefull if you don't have a lot of input_variables in the messages.

In [29]:
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

sys_mesg = SystemMessage(content="You are a helpful assistant.")
human_msg = HumanMessage(content="How are you?")

chat_prompt = ChatPromptTemplate.from_messages([sys_mesg, human_msg])
chat_prompt.format_messages()

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}),
 HumanMessage(content='How are you?', additional_kwargs={}, example=False)]

### Few-shot Prompt Templates
[ref](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/few_shot_examples)

Used when creating prompts with few shot examples. You can load examples from 
- a list of examples
- using `ExampleSelector` which helps you add more logic around selecting examples for the prompt like the `SemanticSimilarityExampleSelector` which selects examples similar to the input.

first lets look at a list of examples. Make sure the examples are a list of `dict`s who's keys match the prompt's input variable.

In [8]:
examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
"""
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
"""
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
"""
  },
  {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

In [11]:
# example prompt is used to format the examples

example_prompt = PromptTemplate.from_template(
    "Question: {question}\n{answer}"
)

print(example_prompt.format(**examples[0]))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



In [13]:
from langchain.prompts import FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}", # will put this at the end
    prefix="",                  # also present if you want
    input_variables=["input"],  # define input vars like any other prompts
)

print(prompt.format(
    input="Who was the father of Mary Ball Washington?"
))

Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: When was the founder of craigslist born?

Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952


Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball W

Now lets use `ExampleSelector` and `SemanticSimilarityExampleSelector` to retrieve examples similar to input.

In [15]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# save examples into a vectorDB
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=OpenAIEmbeddings(),
    vectorstore_cls=Chroma,
    k=2
)

once the DB has been created you can fetch similar ones base on the input_variables of you choice

In [16]:
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples(
    {"question": question}
)
len(selected_examples)

2

In [21]:
# pretty print
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")



question: Who was the maternal grandfather of George Washington?
answer: 
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball



question: Who lived longer, Muhammad Ali or Alan Turing?
answer: 
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali



Pass on the `ExampleSelector` to the few-shot template object

In [22]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector, 
    example_prompt=example_prompt, 
    suffix="Question: {input}", 
    input_variables=["input"]
)

print(prompt.format(input="Who was the father of Mary Ball Washington?"))

Question: Who was the maternal grandfather of George Washington?

Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball


Question: Who lived longer, Muhammad Ali or Alan Turing?

Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali


Question: Who was the father of Mary Ball Washington?


### Few Shot examples for ChatModels

*seems like this is not yet decided - but we want this for ragas

### Format template output

- `format()` - creates a string output with the prompts
- `format_prompt()` - creates chat messages from the prompts. Returns a `PromptValue` which has `to_string()` and `to_messages()`, the later can be used for creating system-human-AI messages for ChatModels.

In [23]:
template = """\
You are a naming consultant for new companies.
What is a good name for a company that makes {product}?
"""

prompt = PromptTemplate.from_template(template)
prompt.format(product="colorful socks")

'You are a naming consultant for new companies.\nWhat is a good name for a company that makes colorful socks?\n'

In [24]:
prompt.format_prompt(product="colorful pants")

StringPromptValue(text='You are a naming consultant for new companies.\nWhat is a good name for a company that makes colorful pants?\n')

In [25]:
sys_msg = SystemMessagePromptTemplate.from_template(
    "You are a helpful assistant that translates {input_language}"
    " to {output_language}"
)
human_msg = HumanMessagePromptTemplate.from_template(
    "{text}"
)


# ChatPromptTemplate has the collection
chat_prompt = ChatPromptTemplate.from_messages([sys_msg, human_msg])

# as a string
chat_prompt.format(
    input_language="English", 
    output_language="French", 
    text="I love programming."
)

'System: You are a helpful assistant that translates English to French\nHuman: I love programming.'

In [27]:
# as messages
chat_prompt.format_prompt(
    input_language="English", 
    output_language="French", 
    text="I love programming."
).to_messages()

[SystemMessage(content='You are a helpful assistant that translates English to French', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

### Partial Prompt Templates

This is when you want to pass a subset of required values to create a new prompt template that expects only the remaining subset of values. The cool thing is that langchain allows you to use strings and functions that return strings as partials.

lets see them in action

In [30]:
from langchain.prompts import PromptTemplate

In [32]:
prompt = PromptTemplate(template="{foo}{bar}", input_variables=["foo", "bar"])

# you only need to pass in foo now
partial_prompt = prompt.partial(foo="foo");
print(partial_prompt.format(bar="baz"))

foobaz


Similarly for functions too.

In [37]:
from datetime import datetime

def _get_date():
    now = datetime.now()
    return now.strftime("%m/%d/%Y")

In [38]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}", 
    input_variables=["adjective", "date"]
);
partial_prompt = prompt.partial(date=_get_date)
print(partial_prompt.format(adjective="funny"))

Tell me a funny joke about the day 07/13/2023


### Custom Prompt Template
[ref](https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/custom_prompt_template)

Langchain offers `PromptTemplate` and `ChatPromptTemplate` but there can be situations where you need your own templates. The examples langchain is uses is a custom template that takes in a function name as input and formats the prompt template to provide the source code of the function. If you building a product to help with devs, this could be very valuable.

In [44]:
import inspect
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

# func to get source
def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

# the custom PromptTemplate
class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the function name as input, and formats the prompt template to provide the source code of the function."""

    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable.")
        return v

    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the language model
        prompt = f"""\
Given the function name and source code, generate an English language explanation of the function.
Function Name: {kwargs["function_name"].__name__}
Source Code:
```python
{source_code}
```
Explanation:
"""
        return prompt

    def _prompt_type(self):
        return "function-explainer"

Now lets ask it to explain the code we just wrote 😉

In [52]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the format() function we wrote above
prompt = fn_explainer.format(
    function_name=FunctionExplainerPromptTemplate.format
)
print(prompt)

Given the function name and source code, generate an English language explanation of the function.
Function Name: predict
Source Code:
```python
    def predict(
        self, text: str, *, stop: Optional[Sequence[str]] = None, **kwargs: Any
    ) -> str:
        if stop is None:
            _stop = None
        else:
            _stop = list(stop)
        return self(text, stop=_stop, **kwargs)

```
Explanation:



but lets see this in action shall we

In [53]:
print(openai.predict(prompt))

This function is called 'predict' and its purpose is to take a string of text, an optional sequence of strings, and any additional keyword arguments as inputs and return a string as an output. If a sequence of strings is provided, the function converts it into a list before returning the output.
